In [1]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
from interaction_inference import simulation
from interaction_inference.constraints import Constraint
from interaction_inference.optimization import Optimization

In [3]:
rng = np.random.default_rng(1)

In [18]:
# simulate sample
data = simulation.simulate_dataset_range_BD(
    name = "test",
    interaction_values = [50, 20, 10, 8, 6, 4, 2, 1, 0.5, 0], #[5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, 0.5, 0],
    cells = 1000,
    rate = 1,
    tqdm_disable = False
)

100%|██████████| 10/10 [00:07<00:00,  1.33it/s]


In [24]:
# downsample
m = 0.75
b = (1 / m) - 1
beta = rng.beta(1, b, size=1000)
data_downsampled = data.downsample("test", beta)

In [25]:
# bootstrap
data_downsampled.compute_moments(tqdm_disable=False)
data_downsampled.bootstrap_probabilities(tqdm_disable=False)
data_downsampled.compute_fm(tqdm_disable=False)

100%|██████████| 10/10 [00:00<00:00, 23.64it/s]


In [26]:
# construct constraints
constraints = Constraint(
    downsampled_probability = True,
    downsampled_CME         = True,
    k_deg_1                 = True,
    k_reg                   = True
)

# optimize
opt = Optimization(data_downsampled, constraints, tqdm_disable=True, print_solution=False)

In [27]:
# construct constraints
constraints_eq = Constraint(
    downsampled_probability = True,
    downsampled_CME         = True,
    k_deg_1                 = True,
    k_reg                   = True,
    equality                = True
)

# optimize
opt_eq = Optimization(data_downsampled, constraints_eq, tqdm_disable=True, print_solution=False)

In [28]:
for i in opt.result_dict.keys():
    print(f"{opt.result_dict[i]['status']}, {opt_eq.result_dict[i]['status']}")

OPTIMAL, OPTIMAL
INFEASIBLE, INFEASIBLE
OPTIMAL, OPTIMAL
INFEASIBLE, INFEASIBLE
INFEASIBLE, INFEASIBLE
INFEASIBLE, INFEASIBLE
OPTIMAL, OPTIMAL
OPTIMAL, OPTIMAL
OPTIMAL, OPTIMAL
OPTIMAL, OPTIMAL
